In [1]:
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import os

In [2]:
types_names = {90:'Ia', 67: '91bg', 52:'Iax', 42:'II', 62:'Ibc', 
               95: 'SLSN', 15:'TDE', 64:'KN', 88:'AGN', 92:'RRL', 65:'M-dwarf',
               16:'EB',53:'Mira', 6:'MicroL', 991:'MicroLB', 992:'ILOT', 
               993:'CART', 994:'PISN',995:'MLString'}

In [3]:
cases = ['50SNIa50SNIbc',
        '50SNIa50SNIax',
        '50SNIa50AGN',
        '50SNIa50SLSN-I',
        '50SNIa50SNII',
        '50SNIa50TDE',
        '50SNIa50KN',
        '50SNIa50SNIa-91bg',
        '68SNIa32SNIbc',
        '68SNIa32SLSN-I',
        '68SNIa32SNIax',
        '75SNIa25SNII',
        '75SNIa25KN',
        '90SNIa10KN',
        '90SNIa10SNIa-91bg',
         '90SNIa10AGN', 
        '90SNIa10SNIax',
         '95SNIa5SNIbc',
    '95SNIa5TDE',
     #'95SNIa5KN',
     '95SNIa5SLSN-I',
     '95SNIa5SNIax',
     '98SNIa2KN',
    '98SNIa2TDE',
    '99SNIa1SNIbc',
    '98SNIa2AGN',
     '99SNIa1AGN',
     '99SNIa1SLSN-I',
    #'fiducial',
     'perfect']

In [18]:
names = []

pop_Ia_all = []
pop_nIa_all = []
perc_Ia_all = []
perc_nIa_all = []

wfit_w_all = []
wfit_wsig_all = []
wfit_om_all = []
wfit_omsig_all = []

stan_w_all = []
stan_wsig_all = []
stan_om_all = []
stan_omsig_all = []

stan_w_all_lowz = []
stan_wsig_all_lowz = []
stan_om_all_lowz = []
stan_omsig_all_lowz = []

other_index = []
other_name = []

for case in cases:

    names.append(case)
    
    pop = {}
    perc = {}

    samples_dir = '/media/RESSPECT/data/PLAsTiCC/for_metrics/posteriors/' + case + '/'

    if case in ['perfect', 'fiducial']:
        fname = '/media/RESSPECT/data/PLAsTiCC/for_metrics/' + case + '_samp.csv'
    else:
        fname = '/media/RESSPECT/data/PLAsTiCC/for_metrics/samples/' + case + '.csv'
    
    data = pd.read_csv(fname)
    stats = np.unique(data['code'].values, return_counts=True)

    if len(stats[0]) > 2:
        raise ValueError('Found more than 2 types in ' + case + '!')
    else:

        fname_cosmo = case + '/results/cosmo_photoIa_plasticc_' + case + '.dat'
        cosmofit = pd.read_csv(fname_cosmo)
        wfit_w_all.append(cosmofit['w'].values[0])
        wfit_wsig_all.append(cosmofit['wsig_marg'].values[0])
        wfit_om_all.append(cosmofit['OM'].values[0])
        wfit_omsig_all.append(cosmofit['OM_sig'].values[0])

        fname_stan = case + '/results/stan_summary_' + case + '.dat'
        op1 = open(fname_stan, 'r')
        lin1 = op1.readlines()
        op1.close()

        for i in range(len(lin1)):
            if lin1[i].split(' ')[0] == 'om':
                c = 3
                found = False
                while not found:
                    if lin1[i].split(' ')[c] != '':
                        stan_om_all.append(lin1[i].split(' ')[c])
                        found=True
                    else:
                        c = c + 1
                    
                d = 8
                found2 = False
                while not found2:
                    if lin1[i].split(' ')[d] != '':
                        stan_omsig_all.append(lin1[i].split(' ')[d])
                        found2 = True
                    else:
                        d = d + 1
            elif lin1[i].split(' ')[0] == 'w':
                c = 3
                found = False
                while not found:
                    if lin1[i].split(' ')[c] != '':
                        stan_w_all.append(lin1[i].split(' ')[c])
                        found=True
                    else:
                        c = c + 1
                    
                d = 8
                found2 = False
                while not found2:
                    if lin1[i].split(' ')[d] != '':
                        stan_wsig_all.append(lin1[i].split(' ')[d])
                        found2 = True
                    else:
                        d = d + 1
    
        fname_stan = case + '/results/stan_summary_' + case + '_lowz_withbias.dat'
        op2 = open(fname_stan, 'r')
        lin2 = op2.readlines()
        op2.close()
    
        for j in range(len(lin2)):
            if lin2[j].split(' ')[0] == 'om':
                stan_om_all_lowz.append(lin2[j].split(' ')[3])
                stan_omsig_all_lowz.append(lin2[j].split(' ')[8])
                
            elif lin2[j].split(' ')[0] == 'w':
                c = 3
                found = False
                while not found:
                    if lin2[j].split(' ')[c] != '':
                        stan_w_all_lowz.append(lin2[j].split(' ')[c])
                        found=True
                    else:
                        c = c + 1
                        
                        
                d = 8
                found2 = False
                while not found2:
                    if lin2[j].split(' ')[d] != '':
                        stan_wsig_all_lowz.append(lin2[j].split(' ')[d])
                        found2 = True
                    else:
                        d = d + 1
                        
         
        pop[stats[0][0]] = stats[1][0]
        perc[stats[0][0]] = round(100 * stats[1][0]/data.shape[0])
        if  len(stats[0]) > 1:
            other_code = [item for item in stats[0] if item !=90][0]
            pop[stats[0][1]] = stats[1][1]
            perc[stats[0][1]] = round(100 * stats[1][1]/data.shape[0])
            
            pop_nIa_all.append(pop[other_code])
            perc_nIa_all.append(perc[other_code])
            other_index.append(other_code)
            other_name.append(types_names[other_code])
        else:
            other_code = '--'
            pop_nIa_all.append(None)
            perc_nIa_all.append(None)
            other_index.append(None)
            other_name.append(None)

        pop_Ia_all.append(pop[90])
        perc_Ia_all.append(perc[90])

        
        
data_all = {}
data_all['case'] = names
data_all['other_name'] = other_name
data_all['other_code'] = other_index
data_all['nIa'] = pop_Ia_all
data_all['nothers'] = pop_nIa_all
data_all['perc_Ia'] = perc_Ia_all
data_all['perc_others'] = perc_nIa_all
data_all['wfit_w'] = wfit_w_all
data_all['wfit_wsig'] = wfit_wsig_all
data_all['wfit_om'] = wfit_om_all
data_all['wfit_omsig'] = wfit_omsig_all
data_all['stan_w'] = stan_w_all
data_all['stan_wsig'] = stan_wsig_all
data_all['stan_om'] = stan_om_all
data_all['stan_omsig'] = stan_omsig_all
data_all['stan_w_lowz'] = stan_w_all_lowz
data_all['stan_wsig_lowz'] = stan_wsig_all_lowz
data_all['stan_om_lowz'] = stan_om_all_lowz
data_all['stan_omsig_lowz'] = stan_omsig_all_lowz

data_all = pd.DataFrame(data_all)

In [20]:
data_all.to_csv('summary_cases.csv', index=False)

In [14]:
lin2[5]

'om   0.31  4.9e-4   0.02   0.27    0.3   0.32   0.33   0.36   2311    1.0\n'

In [99]:
lin2[6].split(' ')[4] 

'-1.7'